<a href="https://colab.research.google.com/github/ForfeitWin/Vanguard-Website/blob/main/vldlp_c_vs_hsCRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, Flatten

# Load the dataset
data = pd.read_csv('/content/sample_data/vldlp_hscrp_data.csv')

# Extract features and target
X = data[['VLDLP_C', 'hsCRP']].values
y = data['Target'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data for CNN input
X_train_cnn = X_train.reshape((X_train.shape[0], 2, 1))
X_test_cnn = X_test.reshape((X_test.shape[0], 2, 1))

# Build the CNN model
input_cnn = Input(shape=(2, 1))
conv1 = Conv1D(64, kernel_size=1, activation='relu')(input_cnn)
flat = Flatten()(conv1)
dense1 = Dense(64, activation='relu')(flat)
output = Dense(1, activation='sigmoid')(dense1)

# Compile the model
model = Model(inputs=input_cnn, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_cnn, y_train, epochs=10, validation_data=(X_test_cnn, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_cnn, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Assuming 'model' is your trained model
model.save('cnn_model.h5')



Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6984 - loss: 0.6304 - val_accuracy: 0.9150 - val_loss: 0.4210
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9521 - loss: 0.3691 - val_accuracy: 0.9200 - val_loss: 0.2553
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9481 - loss: 0.2047 - val_accuracy: 0.9300 - val_loss: 0.1838
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9585 - loss: 0.1380 - val_accuracy: 0.9250 - val_loss: 0.1633
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9535 - loss: 0.1203 - val_accuracy: 0.9250 - val_loss: 0.1517
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9710 - loss: 0.0942 - val_accuracy: 0.9300 - val_loss: 0.1486
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9647 - loss: 0.0998 - val_accuracy: 0.9300 - val_loss: 0.1412
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9588 - loss: 0.1046 - val_accuracy: 0.9450 - val_loss

Test Loss: 0.133920818567276
Test Accuracy: 0.9350000023841858


In [6]:
!pip install dash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

# Load the pre-trained model
# Ensure that you saved the model after training (e.g., model.save('cnn_model.h5'))
model = load_model('cnn_model.h5')

# Load the data and create the scaler (use the same scaler used during training)
data = pd.read_csv('/content/sample_data/vldlp_hscrp_data.csv')
scaler = StandardScaler()
X = data[['VLDLP_C', 'hsCRP']].values
scaler.fit(X)  # Fit the scaler on the training data

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("VLDLP-C and hsCRP Predictor"),

    html.Label("VLDLP-C level"),
    dcc.Input(id='vldlp_input', type='number', value=50),

    html.Label("hsCRP level"),
    dcc.Input(id='hscrp_input', type='number', value=2),

    html.Button('Predict', id='predict-button', n_clicks=0),
    html.Div(id='prediction-output')
])

# Define the callback for prediction
@app.callback(
    Output('prediction-output', 'children'),
    [Input('predict-button', 'n_clicks')],
    [Input('vldlp_input', 'value'), Input('hscrp_input', 'value')]
)
def predict_vldlp_hscrp(n_clicks, vldlp_value, hscrp_value):
    if n_clicks > 0:
        # Prepare the input data
        input_data = np.array([[vldlp_value, hscrp_value]])
        input_data = scaler.transform(input_data)
        input_data = input_data.reshape((input_data.shape[0], 2, 1))

        # Make a prediction
        prediction = model.predict(input_data)
        result = "High Risk" if prediction > 0.5 else "Low Risk"
        return f"Prediction: {result}"
    return ""

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 30.0 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

In [ ]:
%%writefile app.py
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

# Load the pre-trained model
model = load_model('cnn_model.h5')

# Load the data and create the scaler
data = pd.read_csv('vldlp_hscrp_data.csv')
scaler = StandardScaler()
X = data[['VLDLP_C', 'hsCRP']].values
scaler.fit(X)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1("VLDLP-C and hsCRP Predictor"),
    html.Label("VLDLP-C level"),
    dcc.Input(id='vldlp_input', type='number', value=50),
    html.Label("hsCRP level"),
    dcc.Input(id='hscrp_input', type='number', value=2),
    html.Button('Predict', id='predict-button', n_clicks=0),
    html.Div(id='prediction-output')
])

# Define the callback
@app.callback(
    Output('prediction-output', 'children'),
    [Input('predict-button', 'n_clicks')],
    [Input('vldlp_input', 'value'), Input('hscrp_input', 'value')]
)
def predict_vldlp_hscrp(n_clicks, vldlp_value, hscrp_value):
    if n_clicks > 0:
        input_data = np.array([[vldlp_value, hscrp_value]])
        input_data = scaler.transform(input_data)
        input_data = input_data.reshape((input_data.shape[0], 2, 1))

        prediction = model.predict(input_data)
        result = "High Risk" if prediction > 0.5 else "Low Risk"
        return f"Prediction: {result}"
    return ""

if __name__ == '__main__':
    app.run_server(debug=True)



Writing app.py


In [ ]:
!pip freeze


absl-py==1.4.0
accelerate==0.34.2
aiohappyeyeballs==2.4.2
aiohttp==3.10.8
aiosignal==1.3.1
alabaster==0.7.16
albucore==0.0.16
albumentations==1.4.15
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.19.0
astropy==6.1.4
astropy-iers-data==0.2024.9.30.0.32.59
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==24.2.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
beautifulsoup4==4.12.3
bigframes==1.19.0
bigquery-magics==0.2.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.4.3
Bottleneck==1.4.0
bqplot==0.12.43
branca==0.8.0
build==1.2.2
CacheControl==0.14.0
cachetools==5.5.0
catalogue==2.0.10
certifi==2024.8.30
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.87
clarabel==0.9.0
click==8.1.7
cloudpathlib==0.19.0
cloudpickle==2.2.1
cmake==3.30.4
cmdstanpy==1.2.4
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
